In [2]:
import sympy as syp

In [3]:
x, y, z, alpha, beta, theta, phi, lam, epsilon = syp.symbols('x y z alpha beta theta phi lambda epsilon')

In [4]:
dxdt = z*(2*(1-x) - lam*y*(y+1)) - alpha*x*y**2 - theta*x
dydt = alpha/lam * x*y**2 - beta*y**2 + theta/lam * x - phi*y
dzdt = 1/epsilon * (1 - z*(2*(1-x) - lam*y*(y+1)))

In [5]:
J = syp.Matrix([dxdt, dydt, dzdt]).jacobian([x, y, z])

In [6]:
eigenvalues = J.eigenvals()

In [7]:
f = beta*y**2 + phi*y - 1/lam
y0_2, y0 = syp.solve(f, y)

In [8]:
y0

(-lambda*phi + sqrt(lambda*(4*beta + lambda*phi**2)))/(2*beta*lambda)

In [9]:
y0_2

(-lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))/(2*beta*lambda)

In [10]:
J

Matrix([
[       -alpha*y**2 - theta - 2*z, -2*alpha*x*y + z*(-lambda*y - lambda*(y + 1)),          -lambda*y*(y + 1) - 2*x + 2],
[alpha*y**2/lambda + theta/lambda,           2*alpha*x*y/lambda - 2*beta*y - phi,                                    0],
[                     2*z/epsilon,       -z*(-lambda*y - lambda*(y + 1))/epsilon, (lambda*y*(y + 1) + 2*x - 2)/epsilon]])

In [11]:
x0 = syp.simplify(1/(alpha*y0**2 + theta))

In [12]:
x0

4*beta**2*lambda**2/(alpha*(lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))**2 + 4*beta**2*lambda**2*theta)

In [13]:
z0 = syp.simplify(1/(2*(1-x0) - lam*y0*(y0+1)))

In [14]:
z0

-1/(8*beta**2*lambda**2/(alpha*(lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))**2 + 4*beta**2*lambda**2*theta) - 2 - (1 - (lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))/(2*beta*lambda))*(lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))/(2*beta))

In [15]:
fixpt = syp.Matrix([x0, y0, z0])
fixpt

Matrix([
[                                                                                                                                                    4*beta**2*lambda**2/(alpha*(lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))**2 + 4*beta**2*lambda**2*theta)],
[                                                                                                                                                                                              (-lambda*phi + sqrt(lambda*(4*beta + lambda*phi**2)))/(2*beta*lambda)],
[-1/(8*beta**2*lambda**2/(alpha*(lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))**2 + 4*beta**2*lambda**2*theta) - 2 - (1 - (lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))/(2*beta*lambda))*(lambda*phi - sqrt(lambda*(4*beta + lambda*phi**2)))/(2*beta))]])

In [16]:
J_sub = syp.simplify(J.subs({x: x0, y: y0, z: z0}))

In [21]:
M = J_sub.subs({alpha: 10, beta: 50, theta: 1, phi: 50, lam: 0.1, epsilon: 1000})

In [22]:
M.eigenvals()

{-39.4979446596911: 1, -7.06134573650454: 1, -0.000134149389703059: 1}